Parallel Programming
====

The goal is to design parallel programs that are flexible, efficient and simple.

**Step 0**: Start by profiling a serial program to identify bottlenecks

**Step 1**: Are there for opportunities for parallelism?

- Can tasks be performed in parallel?
    - Function calls
    - Loops
- Can data be split and operated on in parallel?
    - Decomposition of arrays along rows, columns, blocks
    - Decomposition of trees into sub-trees
- Is there a pipeline with a sequence of stages?
    - Data preprocessing and analysis
    - Graphics rendering

**Step 2**: What is the nature of the parallelism?

- Linear
    - Embarrassingly parallel programs
- Recursive
    - Adaptive partitioning methods

**Step 3**: What is the granularity?

- 10s of jobs
- 1000s of jobs

**Step 4**: Choose an algorithm

- Organize by tasks
    - Task parallelism
    - Divide and conquer

- Organize by data
    - Geometric decomposition
    - Recursive decomposition

- Organize by flow
    - Pipeline
    - Event-based processing

**Step 5**: Map to program and data structures

- Program structures
    - Single program multiple data (SPMD)
    - Master/worker
    - Loop parallelism
    - Fork/join
- Data structures 
    - Shared data
    - Shared queue
    - Distributed array

**Step 6**: Map to parallel environment

- Multi-core shared memory
    - Cython with OpenMP
    - multiprocessing
    - IPython.cluster
- Multi-computer
    - IPython.cluster
    - MPI
    - Hadoop / Spark
- GPU
    - CUDA
    - OpenCL

**Step 7**: Execute, debug, tune in parallel environment

Embarrassingly parallel programs
----

Many statistical problems are embarrassingly parallel and can be easily decomposed into independent tasks or data sets. Here are several examples:

- Monte Carlo integration
- Multiple chains of MCMC
- Bootstrap for confidence intervals
- Power calculations by simulation
- Permutation-resampling tests 
- Fitting same model on multiple data sets

Other problems are serial at small scale, but can be parallelized at large scales. For example, EM and MCMC iterations are inherently serial since there is a dependence on the previous state, but within a single iteration, there can be many thousands of density calculations (one for each data point to calculate the likelihood), and this is an embarrassingly parallel problem within a single iteration. 

These "low hanging fruits" are great because they offer a path to easy parallelism with minimal complexity.

Executing parallel code
----

**The bigger the problem, the more scope there is for parallelism**

**Amhdahls' law** says that the speedup from parallelization is bounded by the ratio of parallelizable to irreducibly serial code in the algorithm. However, for big data analysis, **Gustafson's Law** is more relevant. This says that we are nearly always interested in increasing the size of the parallelizable bits, and the ratio of parallelizable to irreducibly serial code is not a static quantity but depends on data size. For example, Gibbs sampling has an irreducibly serial nature, but for large samples, each iteration may be able perform PDF evaluations in parallel for zillions of data points.

Threading
----

The Python interpreter only allows one thread to execute at any one time. This is known as the Global Interpreter Lock (GIL) and generally makes the use of threads an ineffective strategy for CPU-bound tasks. However, threads are useful in I/O bound tasks such as reading/writing/downloading files since threads give up their lock once an I//O task has been initiated. 

### CPU-bound task

**Estimating $\pi$ using Monte Carlo integration**

This is clearly a toy example, but the template can be used for most embarrassingly parallel problems and is a good example of a CPU-bound task.

In [8]:
def mc_pi(n):
    s = 0
    for i in range(n):
        x = np.random.uniform(-1, 1)
        y = np.random.uniform(-1, 1)
        if (x**2 + y**2) < 1:
            s += 1
    return 4*s/n

In [5]:
n = int(1e7)

In [10]:
%%time
mc_pi(n)

CPU times: user 15.9 s, sys: 12.8 ms, total: 15.9 s
Wall time: 15.9 s


3.1421176

#### Optimizations without parallelism

Review of techniques to accelerate Python code.

In [14]:
def mc_pi1(n):
    x = np.random.uniform(-1, 1, (n,2))
    return 4*np.sum((x**2).sum(1) < 1)/n

In [15]:
%%time 
mc_pi1(n)

CPU times: user 818 ms, sys: 273 ms, total: 1.09 s
Wall time: 1.12 s


3.1409736000000001

In [16]:
from numba import jit, njit

In [17]:
@njit()
def mc_pi2(n):
    s = 0
    for i in range(n):
        x = np.random.uniform(-1, 1)
        y = np.random.uniform(-1, 1)
        if (x**2 + y**2) < 1:
            s += 1
    return 4*s/n

In [18]:
%%time
mc_pi2(n)

CPU times: user 561 ms, sys: 7.39 ms, total: 568 ms
Wall time: 570 ms


3.1423348

#### Cython

It is a little more work to optimize the Cython version. We do not use numpy.random as that is optimized for generating arrays rather than single numbers. Instead we use the GNU Scientific Library (wrapped for us in the package cython_gsl) to draw uniform random variates.

```
pip install cythongsl
```

In [35]:
%%cython -a -lgsl

import cython
from cython_gsl cimport *

@cython.cdivision(True) 
def mc_pi3(int n):
    cdef gsl_rng_type * T
    cdef gsl_rng * r
    cdef double s = 0.0
    cdef double x, y
    cdef int i

    gsl_rng_env_setup()

    T = gsl_rng_default
    r = gsl_rng_alloc (T)

    for i in range(n):
        x = 2*gsl_rng_uniform(r) - 1
        y = 2*gsl_rng_uniform(r)- 1
        if (x**2 + y**2) < 1:
            s += 1
    return 4*s/n

In [36]:
%%time
mc_pi3(n)

CPU times: user 316 ms, sys: 1.34 ms, total: 317 ms
Wall time: 316 ms


3.1414584